In [ ]:
!pip install isbnlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 1.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import isbnlib
import matplotlib.pyplot as plt
from scipy.cluster.vq import kmeans, vq
from pylab import plot, show
from matplotlib.lines import Line2D
import matplotlib.colors as mcolors
from sklearn.cluster import KMeans
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from progressbar import ProgressBar
from sklearn.metrics import silhouette_score

import warnings
warnings.filterwarnings("ignore")
print('Libraries imported!')

Libraries imported!


In [ ]:
df = pd.read_csv(r'/content/drive/MyDrive/ML project/book_info.csv', error_bad_lines = False)

df

,Unnamed: 0,id,Book_Name,Writer_Name,Category,Rating,Review,Site,sentiment,label,Book_name_without_spaces
0,0,96523,মুনতাখাব হাদীস,মাওলানা মোঃ সাখাওয়াত উল্লাহ,হাদিস ও সুন্নাত,1,(১) বই টি শত শত জাল হাদিসে ভরপুর । (২) দ্বীন ...,Rokomari,negative,0,মুনতাখাবহাদীস
1,1,39118,মেঘাচ্ছন্ন ঈমান (পেপারব্যাক),শাইখ আব্দুল মালিক আল কাসিম,"ঈমান, আক্বিদা ও তাওবাহ",1,"বই রিভিউ : মেঘাচ্ছন্ন ঈমান পাঠ অনুভূতি: ""একটি...",Rokomari,negative,0,মেঘাচ্ছন্নঈমান(পেপারব্যাক)
2,2,141682,সামষ্টিক অর্থনীতি (পার্স কোর্স দ্বিতীয় বর্ষের...,সুকেশ চন্দ্র জোয়ারদার,ডিগ্রি,1,পড়তে চাই,Rokomari,negative,0,সামষ্টিকঅর্থনীতি(পার্সকোর্সদ্বিতীয়বর্ষের-৩য়পত্...
3,3,136770,ছাত্রলীগের ইতিহাস বাংলাদেশের ইতিহাস (১৯৪৮-১৯৭...,সাইফুর রহমান সোহাগ,প্রসঙ্গ: বাংলাদেশের রাজনীতি,1,ঠিক আছে,Rokomari,negative,0,ছাত্রলীগেরইতিহাসবাংলাদেশেরইতিহাস(১৯৪৮-১৯৭০প্রথ...
4,4,82740,"এসএসসি বিজ্ঞান মেইড ইজি প্রশ্নপত্র, সকল শিক্ষ...",পাঞ্জেরী সম্পাদনা পর্ষদ (সম্পাদক),নবম ও দশম (এসএসসি): বিজ্ঞান বিভাগ পাঠ্য সহায়...,1,এটি এসএসসি শিক্ষার্থীর 2020 এর জন্য একটি পরীক্...,Rokomari,negative,0,"এসএসসিবিজ্ঞানমেইডইজিপ্রশ্নপত্র,সকলশিক্ষাবোর্ড,..."
...,...,...,...,...,...,...,...,...,...,...,...
6306,6306,21943,"তিমির প্রেম, সপ্ত আতঙ্ক, লর্না ডুন (পেপারব্যা...",আর. ডি. ব্ল্যাকমোর,"রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার, ও অ্যাড...",5,এই সংগ্রহটি তিনটি দুর্দান্ত ক্লাসিক উপন্যাস নি...,Rokomari,positive,2,"তিমিরপ্রেম,সপ্তআতঙ্ক,লর্নাডুন(পেপারব্যাক)তিনটি..."
6307,6307,51194,Guinness World Records 2020 (Hardcover),Guinness World Records (Editor),"Foreign Language Books: Essay, Articles & Res...",5,ভাল বই,Rokomari,positive,2,GuinnessWorldRecords2020(Hardcover)
6308,6308,35327,খুন (হার্ডকভার),আহমেদ ফারুক,সমকালীন উপন্যাস,5,সম্পূর্ণ প্রাপ্তবয়স্কদের বই। তবে দুর্দান্ত গল...,Rokomari,positive,2,খুন(হার্ডকভার)
6309,6309,8262,একাত্তর এবং মুক্তিযোদ্ধা পরিবার (হার্ডকভার),আওলিয়া খানম,"মুক্তিযুদ্ধের ডায়েরি, চিঠি ও স্মৃতিচারণ",5,"এটি ""সাহিত্য কথা"" প্রকাশনী হতে প্রকাশিত একটি অ...",Rokomari,positive,2,একাত্তরএবংমুক্তিযোদ্ধাপরিবার(হার্ডকভার)


In [ ]:
df = df.drop(columns=['Unnamed: 0'])

In [ ]:
df

,id,Book_Name,Writer_Name,Category,Rating,Review,Site,sentiment,label,Book_name_without_spaces
0,96523,মুনতাখাব হাদীস,মাওলানা মোঃ সাখাওয়াত উল্লাহ,হাদিস ও সুন্নাত,1,(১) বই টি শত শত জাল হাদিসে ভরপুর । (২) দ্বীন ...,Rokomari,negative,0,মুনতাখাবহাদীস
1,39118,মেঘাচ্ছন্ন ঈমান (পেপারব্যাক),শাইখ আব্দুল মালিক আল কাসিম,"ঈমান, আক্বিদা ও তাওবাহ",1,"বই রিভিউ : মেঘাচ্ছন্ন ঈমান পাঠ অনুভূতি: ""একটি...",Rokomari,negative,0,মেঘাচ্ছন্নঈমান(পেপারব্যাক)
2,141682,সামষ্টিক অর্থনীতি (পার্স কোর্স দ্বিতীয় বর্ষের...,সুকেশ চন্দ্র জোয়ারদার,ডিগ্রি,1,পড়তে চাই,Rokomari,negative,0,সামষ্টিকঅর্থনীতি(পার্সকোর্সদ্বিতীয়বর্ষের-৩য়পত্...
3,136770,ছাত্রলীগের ইতিহাস বাংলাদেশের ইতিহাস (১৯৪৮-১৯৭...,সাইফুর রহমান সোহাগ,প্রসঙ্গ: বাংলাদেশের রাজনীতি,1,ঠিক আছে,Rokomari,negative,0,ছাত্রলীগেরইতিহাসবাংলাদেশেরইতিহাস(১৯৪৮-১৯৭০প্রথ...
4,82740,"এসএসসি বিজ্ঞান মেইড ইজি প্রশ্নপত্র, সকল শিক্ষ...",পাঞ্জেরী সম্পাদনা পর্ষদ (সম্পাদক),নবম ও দশম (এসএসসি): বিজ্ঞান বিভাগ পাঠ্য সহায়...,1,এটি এসএসসি শিক্ষার্থীর 2020 এর জন্য একটি পরীক্...,Rokomari,negative,0,"এসএসসিবিজ্ঞানমেইডইজিপ্রশ্নপত্র,সকলশিক্ষাবোর্ড,..."
...,...,...,...,...,...,...,...,...,...,...
6306,21943,"তিমির প্রেম, সপ্ত আতঙ্ক, লর্না ডুন (পেপারব্যা...",আর. ডি. ব্ল্যাকমোর,"রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার, ও অ্যাড...",5,এই সংগ্রহটি তিনটি দুর্দান্ত ক্লাসিক উপন্যাস নি...,Rokomari,positive,2,"তিমিরপ্রেম,সপ্তআতঙ্ক,লর্নাডুন(পেপারব্যাক)তিনটি..."
6307,51194,Guinness World Records 2020 (Hardcover),Guinness World Records (Editor),"Foreign Language Books: Essay, Articles & Res...",5,ভাল বই,Rokomari,positive,2,GuinnessWorldRecords2020(Hardcover)
6308,35327,খুন (হার্ডকভার),আহমেদ ফারুক,সমকালীন উপন্যাস,5,সম্পূর্ণ প্রাপ্তবয়স্কদের বই। তবে দুর্দান্ত গল...,Rokomari,positive,2,খুন(হার্ডকভার)
6309,8262,একাত্তর এবং মুক্তিযোদ্ধা পরিবার (হার্ডকভার),আওলিয়া খানম,"মুক্তিযুদ্ধের ডায়েরি, চিঠি ও স্মৃতিচারণ",5,"এটি ""সাহিত্য কথা"" প্রকাশনী হতে প্রকাশিত একটি অ...",Rokomari,positive,2,একাত্তরএবংমুক্তিযোদ্ধাপরিবার(হার্ডকভার)


In [ ]:
books_df = df
numberOfBooks = books_df.shape[0]

print(numberOfBooks)

# Create a DataFrame with 100 columns
columns = ['user' + str(i) for i in range(1, 101)]

data = np.zeros((numberOfBooks, 100))

df1 = pd.DataFrame(data, columns=columns)

# Add a primary key column 'book_id'
# books_df['id']
df1['book_id'] = books_df['id']

df1 = df1[['book_id'] + columns]

df1

6311


,book_id,user1,user2,user3,user4,user5,user6,user7,user8,user9,...,user91,user92,user93,user94,user95,user96,user97,user98,user99,user100
0,96523,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,39118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,141682,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,136770,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,82740,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6306,21943,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6307,51194,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6308,35327,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6309,8262,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import random

ratings = books_df['Rating']

for index, rating in enumerate(ratings):
  ratedBy = random.randint(1, 100)
  people = random.sample(range(1, 101), ratedBy)
  totalVal = ratedBy * rating
  cnt = 0

  for person in people:
    x = random.randint(1, 5)
    stri = 'user'
    stri += str(person)

    if((totalVal - x) >= (ratedBy-cnt-1)) and ((totalVal-x) <= (ratedBy-cnt-1)*5):
      df1[stri][index] = x
      totalVal -= x
    elif(totalVal - x) < (ratedBy-cnt-1):
      df1[stri][index] = 1
      totalVal -= 1
    else:
      df1[stri][index] = 5
      totalVal -= 5
    cnt += 1;

In [ ]:
for index, row in df1.iterrows():
  sum = 0
  cnt1 = 0
  for idx, val in enumerate(row):
    if(idx == 0):
      continue
    if(val != 0):
       cnt1+=1
    sum += val
  print(sum)
  print(cnt1)
  print(sum/cnt1)

In [ ]:
from google.colab import files
df1.to_csv('user_info.csv', encoding = 'utf-8-sig')
files.download('user_info.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>